In [1]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score

In [2]:
from sklearn.model_selection import train_test_split
import category_encoders

In [4]:
features = pd.read_csv("features_08.csv")
kaggle = pd.read_csv('trocafone_kaggle_test.csv')
labels = pd.read_csv('labels_training_set.csv')

In [5]:
training = labels.set_index('person').join(features.set_index('person'))
train_labels = training['label']
train_data = training.drop('label',axis=1)

In [6]:
predict_data = kaggle.set_index('person').join(features.set_index('person'))

In [7]:
encoder = category_encoders.TargetEncoder(smoothing=True)
train_data_enc = encoder.fit_transform(np.array(train_data),np.array(train_labels))
predict_data_enc = encoder.transform(np.array(predict_data))

In [8]:
xtrain,xtest,ytrain,ytest = train_test_split(train_data_enc,np.array(train_labels))

In [9]:
train = xgb.DMatrix(data=xtrain,label=ytrain)
test = xgb.DMatrix(data=xtest,label=ytest)

data = xgb.DMatrix(data=train_data_enc,label=np.array(train_labels))
pred = xgb.DMatrix(data=predict_data_enc)
params = {'colsample_bytree': 0.9,
 'gamma': 2,
 'learning_rate': 0.016238752458245277,
 'max_depth': 13,
 'min_child_weight': 5,
 'n_estimators': 100,
 'subsample': 0.7847190225361189,
 'eval_metric':'auc'}

In [ ]:
bst = xgb.train(params,train,300,[(train,'train'),(test,'eval')])

In [11]:
preds = bst.predict(test)

In [12]:
roc_auc_score(ytest,preds)

0.848208271191358

In [13]:
bst = xgb.train(params,data,200)

[12:45:36] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 84 extra nodes, 330 pruned nodes, max_depth=13
[12:45:36] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 50 extra nodes, 364 pruned nodes, max_depth=12
[12:45:36] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 68 extra nodes, 440 pruned nodes, max_depth=10
[12:45:36] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 66 extra nodes, 344 pruned nodes, max_depth=12
[12:45:36] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 58 extra nodes, 378 pruned nodes, max_depth=12
[12:45:36] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 74 extra nodes, 304 pruned nodes, max_depth=11
[12:45:36] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:

In [14]:
xg_prediction = bst.predict(pred)

## Combinando con catboost

In [15]:
import catboost as cb

In [16]:
best={'depth': 3,
 'iterations': 1000,
 'learning_rate': 0.03,
 'l2_leaf_reg': 3,
 'border_count': 20,
 'thread_count': 4}

In [17]:
cb_classifier = cb.CatBoostClassifier(**best)

In [18]:
cb_classifier.load_model('preentrenados/cb_f08')

In [19]:
cb_prediction = cb_classifier.predict_proba(predict_data)[:,1]

In [20]:
prediction = 0.9*cb_prediction+0.1*xg_prediction

In [21]:
submit = kaggle.set_index('person')

In [22]:
submit['label'] = prediction

In [23]:
submit.to_csv('submit.csv')

In [53]:
submit = kaggle.set_index('person')

In [54]:
submit['label'] = xg_prediction

In [55]:
submit.to_csv('submit1.csv')